In [13]:
import os
from PIL import Image
import numpy as np

openslide_path = r'C:\Users\Donald\Documents\openslide-win64-20230414\bin'

os.environ['PATH'] = openslide_path + ";" + os.environ['PATH']
# from openslide import OpenSlide


if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(openslide_path):
        import openslide
else:
    import openslide
from openslide import OpenSlide

You need to do get crop data before running this code. I use a matlab function I made with the 2.5x or 10x tiff images.

Function is here: \\10.99.68.178\andreex\students\Donald Monkey fetus\codes\monkey\crop1024_monkey.m

In [14]:
pth = r'C:\Users\Donald\Desktop\temp'
pth_mat = r'C:\Users\Donald\Desktop\temp\1x\cropped\matfiles'

In [15]:
# Get a list of MAT files and their base names (without extensions)
mat_files = [os.path.splitext(file)[0] for file in os.listdir(pth_mat) if file.endswith('.mat')]

# Get a list of NDPI images and their base names (without extensions)
ndpi_files = [os.path.splitext(file)[0] for file in os.listdir(pth) if file.endswith('.ndpi')]

# Find the matching files based on their base names
matching_files = set(mat_files).intersection(ndpi_files)

WSIs = sorted([filename + '.ndpi' for filename in matching_files])

crop_mats = sorted([filename + '.mat' for filename in mat_files])

In [16]:
crop_mats

['fallopian_tubes_AJER376_211.mat']

In [17]:
WSIs

['fallopian_tubes_AJER376_211.ndpi']

In [28]:
res20x=0.4416
res2_5x=8

ratio = res2_5x/res20x


outpth = os.path.join(pth,'tiles512')
if not os.path.exists(outpth):
    os.mkdir(outpth)

In [29]:
print(ratio)

18.115942028985508


In [26]:
outpth = r'C:\Users\Donald\Desktop\temp\tiles512'
outpth

'C:\\Users\\Donald\\Desktop\\temp\\tiles512'

In [21]:
crop_mats

['fallopian_tubes_AJER376_211.mat']

In [22]:
WSIs

['fallopian_tubes_AJER376_211.ndpi']

In [37]:
from scipy.io import loadmat

file_format = '.tif'

for i, file in enumerate(crop_mats):
    mat_file_name = os.path.join(pth_mat, file)

    WSI_path = os.path.join(pth, WSIs[i])

    f_out_name = f'{os.path.join(os.path.join(outpth,WSIs[i][:-5]))}{file_format}'

    if os.path.exists(f_out_name):
        print(f'Skipping cropping for {WSIs[i]} ({i+1}/{len(crop_mats)})... Already exists')

    else:
        print(f'Cropping {WSIs[i]} ({i+1}/{len(crop_mats)})')

        # print(mat_file_name)
        # print(WSI_path)

        WSI_slide = openslide.OpenSlide(WSI_path)

        wsi_width, wsi_height = WSI_slide.level_dimensions[0]

        data = loadmat(mat_file_name)

        # crop_width = int(data['crop_width']*ratio)
        # crop_height = int(data['crop_height']*ratio)


        crop_width = 512
        crop_height = 512
        crop_x = int(data['crop_x']*ratio)-50
        crop_y = int(data['crop_y']*ratio)+120

        lower_x = crop_x
        upper_y = crop_y
        upper_x = crop_x + crop_width
        lower_y = crop_y - crop_height

        # print(crop_x)
        # print(crop_y)
        # print(crop_width)
        # print(crop_height)

        region = WSI_slide.read_region((crop_x, crop_y), 0, (crop_width, crop_height)) # level = 0 in middle
        region_image = Image.new('RGB', region.size)
        region_image.paste(region, (0, 0))

        region_image.save(f_out_name)
        WSI_slide.close()


Cropping fallopian_tubes_AJER376_211.ndpi (1/1)


In [38]:
crop_x


19818

In [39]:
crop_y

20134